In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle 
from keras.models import load_model

In [3]:
model = load_model('Model.h5')

with open('Geography_encoder.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('Gender_encoder.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('Scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [55]:
# Example input data
input_data = {
    "CreditScore": 500,
    "Geography": "France",  # Represent this value as 0, 1, or 2 based on encoding for your model
    "Gender": 'Male',            # 0 for Male, 1 for Female
    "Age": 40,
    "Tenure": 1,
    "Balance": 0,
    "NumOfProducts": 2,
    "HasCrCard": 0,
    "IsActiveMember": 0,
    "EstimatedSalary": 100000.50
}

In [56]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\hp\.conda\envs\PY3_10\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [57]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,France,Male,40,1,0,2,0,0,100000.5


In [58]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,France,1,40,1,0,2,0,0,100000.5


In [59]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,500,1,40,1,0,2,0,0,100000.5,1.0,0.0,0.0


In [60]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-1.57375827,  0.91324755,  0.10479359, -1.38944225, -1.21847056,
         0.80843615, -1.54035103, -1.02583358, -0.00749   ,  1.00150113,
        -0.57946723, -0.57638802]])

In [61]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 18ms/step


array([[0.04750243]], dtype=float32)

In [62]:
prediction_proba = prediction[0][0]

In [63]:
prediction_proba

0.04750243

In [ ]:
if prediction_proba > 0.15:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


: 